In [4]:
import sys, os
import ujson as json
from pathlib import Path 
HOME = "/rds/general/user/dl2119/home/ICLR_Interp" # Path(os.environ["PATH_INTP_FOLDER"])
os.environ["PATH_INTP_FOLDER"] = HOME
HOME = Path(HOME)
sys.path.append(str(HOME))
sys.path.append(str(HOME / "DataGeneration"))
sys.path.append(str(HOME / "ModelGeneration"))

from save_load_data import load_fast
from save_load_model import save_nn
from nn_train import train

import torch
import torchvision
import torch.optim as optim
from torch import Tensor

import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Module

from torch.utils.data import DataLoader

import matplotlib.pyplot as plt 

from scipy.interpolate import RBFInterpolator

from Functions import Schwefel
import numpy as np

import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def normalize(X_data, y_data):
    x_max, x_min = np.max(X_data), np.min(X_data)
    y_max, y_min = np.max(y_data), np.min(y_data)
    X_data = (X_data - (x_max + x_min)/2.) * 5. / ((x_max - x_min)/2.)
    y_data = (y_data - (y_max + y_min)/2.) * 5. / ((y_max - y_min)/2.)

    return X_data, y_data

In [4]:
funcs = ["Schwefel", "Ackley", "Michi", "Rosenbrock"]
dims = [1, 2, 3, 4, 6, 8, 10]
Ns = [1e4] # [1e2, 1e3, 1e4, 1e5, 1e6]
data_gen_methods = ["UniformRandom", "UniformGrid", "HessSampling"]

for func_name in funcs:
    print(func_name)
    for dim in dims:
        if dim == 1 and func_name == "Rosenbrock":
            continue
        print(dim)
        for N in Ns:
            print(N)
            for data_gen_method in data_gen_methods:
                start_time = time.time()
                X_data, y_data = load_fast(func_name, dim, N, data_gen_method)
                X_data, y_data = normalize(X_data, y_data)
#                 rbf = RBFInterpolator(X_data, y_data) #, neighbors=2*len(X_data))
                print(time.time() - start_time)

                    

Schwefel
1
100000.0
(100000,)
0.003522157669067383
(100000,)
0.0022439956665039062
(100000,)
0.0017611980438232422
2
100000.0
(100000,)
0.002794027328491211
(100000,)
0.0030455589294433594
(100000,)
0.0020973682403564453
3
100000.0
(100000,)
0.0027649402618408203
(100000,)
0.002672910690307617
(100000,)
0.002343416213989258
4
100000.0
(100000,)
0.0033631324768066406
(100000,)
0.0033490657806396484
(100000,)
0.0028400421142578125
6
100000.0
(100000,)
0.0047054290771484375
(100000,)
0.004723548889160156
(100000,)
0.0041103363037109375
8
100000.0
(100000,)
0.005838871002197266
(100000,)
0.005870819091796875
(100000,)
0.005392789840698242
10
100000.0
(100000,)
0.00699615478515625
(100000,)
0.008515357971191406
(100000,)
0.008860111236572266
Ackley
1
100000.0
(100000,)
0.0031616687774658203
(100000,)
0.002045154571533203
(100000,)
0.0015361309051513672
2
100000.0
(100000,)
0.0031440258026123047
(100000,)
0.002385377883911133
(100000,)
0.0019609928131103516
3
100000.0
(100000,)
0.00257802009

In [6]:
func_name = "Schwefel"
dim = 1
N = 1e5
data_gen_method = "HessSampling"
X_data, y_data = load_fast(func_name, dim, N, data_gen_method)
X_data, y_data = normalize(X_data, y_data)

In [10]:
X_test = np.random.uniform(low=np.min(X_data), high=np.max(X_data), size=(1000, dim))

In [11]:
# start_time = time.time()
# rbf1 = RBFInterpolator(X_data, y_data) #, neighbors=2*len(X_data))
# print(time.time() - start_time)
start_time = time.time() 
rbf2 = RBFInterpolator(X_data, y_data, neighbors=1e3)
print(time.time() - start_time)

0.019396066665649414


In [12]:
# start_time = time.time()
# y1 = rbf1(X_test)
# print(time.time() - start_time)
start_time = time.time()
y2 = rbf2(X_test)
print(time.time() - start_time)

# np.linalg.norm(y_data - y2)

24.99647569656372


In [13]:
a = np.array([1, 2])

In [14]:
list(a)

[1, 2]